In [14]:
from pprint import pprint

from selenium import webdriver
from bs4 import BeautifulSoup

In [15]:
as_show_link = 'https://www.adultswim.com/videos/ghost-in-the-shell'


In [16]:
# Create the browser

options = webdriver.ChromeOptions()
options.add_argument('headless')
browser = webdriver.Chrome(executable_path='../../utilities/chromedriver', chrome_options=options)

In [17]:
# Make the request, gather the response

browser.get(as_show_link)
html = browser.page_source
browser.quit()

In [23]:
# Parse the response
# Break into seasons
soup = BeautifulSoup(html, 'html.parser')

seasons = soup.find_all(name='div', attrs={'itemprop':'containsSeason'})
print(len(seasons))

2


In [18]:




episodes = soup.find_all('div', class_='_29ThWwPi')

episode_guide = {}

for index, episode in enumerate(episodes):
    episode_data = {}
    meta_tags = episode.find_all('meta')
    for meta_data in meta_tags:
        prop = meta_data.get('itemprop')
        content = meta_data.get('content')
        if prop and content:
            episode_data[prop] = content
    episode_guide[f'episode_{index + 1}'] = episode_data

In [20]:
pprint(episode_guide)

 of the trial, it becomes '
                               'clear that who the attorney is really after is '
                               'not Togusa, but all of Section 9!',
                'duration': 'T24M42S',
                'episodeNumber': '10',
                'expires': '2036-01-31T05:00:00.000Z',
                'thumbnailUrl': 'https://media.cdn.adultswim.com/uploads/20200305/thumbnails/2_2035111235-gits2_2ndgig_010_air_cid-37NNP.jpg',
                'uploadDate': '2019-05-16T15:00:00.000Z'},
 'episode_37': {'contentRating': 'TV-14',
                'datePublished': '2006-02-12T05:30:00.000Z',
                'description': 'While field testing new recruits, the Major '
                               'finds herself hacked, and an old woman tells '
                               'her a story from her past.\n'
                               'Watch more Ghost in the Shell on [adult swim].',
                'duration': 'T24M42S',
                'episodeNumber': '11',
       